In [1]:
import pandas as pd
import numpy as np
import netCDF4 as nc

In [2]:
# 处理 CSV 文件的函数
def process_csv(file_path,init_time = '2024-03-27 15:30:21',interval = 2):
    df = pd.read_csv(file_path,encoding='gb2312')
    # 检测异常
    index_temp = (df['温度'] < 40.0)&(df['温度'] > -2.5)
    df = df[index_temp]
    index_pres = df['深度'] > 0
    df = df[index_pres]
    index_cond = (df['电导率'] < 70.0)&(df['电导率'] > 0)
    df = df[index_cond]
    # 计算时间
    from datetime import datetime, timedelta 
    n = int(df['小浮子序列号'][0])
    init_time =  datetime.strptime(init_time, '%Y-%m-%d %H:%M:%S')
    time = init_time + timedelta(hours=interval*n)
    time = int(time.strftime('%Y%m%d%H%M%S'))
    df['时间'] = time
    return df

In [3]:
file_path = r'C:\Users\lenovo\Desktop\FOP\FOP_FB_001_Pro003_202303270530.csv'
init_time = '2024-03-27 15:30:21'; interval = 2 
df = process_csv(file_path,init_time,interval)

In [4]:
df.head()

,小浮子序列号,平台号,温度,深度,电导率,时间
0,3,1,23.238,2408.6,34.99,20240327213021
1,3,1,23.238,1808.6,34.99,20240327213021
2,3,1,23.238,1708.6,34.99,20240327213021
3,3,1,23.238,1608.6,34.99,20240327213021
4,3,1,23.238,1508.6,34.99,20240327213021


In [1]:
def nc_generate(df,new_path,file_path,longitude = 111.1234,latitude =23.1234,  lon2 = 105.21,lat2 = 15.11,ReleaseTime = '2022-12-11 14:03:07 UTC',ReleaseLongitude = '111.8444°E',ReleaseLatitude = '17.4169°N',Comment = 'quality control data',information ='',interval = 2) :
    # FOP_Type:'BS' lon2:105.21 lat2:15.11 deploy_time:202303270530(整数就可以，字符串也可以)
    name = file_path.split('\\')[-1] # 'AWG_HYY_003_202303270530.csv'
    FOP_Type = name.split('_')[1]
    deviceID = name.split('_')[2]
    
    time = df['时间'][0]
    longitude =np.round(longitude,4); latitude =np.round(latitude,4); 
    import seawater
    profile_pressure = df['深度']; profile_pressure = np.round(profile_pressure,3); 
    profile_temperature = df['温度']; profile_temperature = np.round(profile_temperature,4); 
    profile_conductivity = df['电导率']; profile_conductivity = np.round(profile_conductivity,4); 
    seawater_salinity = seawater.eos80.salt(profile_conductivity/ 42.914, profile_temperature, profile_pressure); 
    seawater_salinity = np.round(seawater_salinity,4); 

    seawater_density = seawater.dens(seawater_salinity, profile_temperature, profile_pressure)
    seawater_sound_velocity = seawater.svel(seawater_salinity, profile_temperature, profile_pressure)
    seawater_density = np.round(seawater_density,4); seawater_sound_velocity = np.round(seawater_sound_velocity,4); 

    num_row = df.shape[0] # 获取数据行数，为 node_dim准备
    # 创建nc文件
    n = int(df['小浮子序列号'][0]); sn = '{:03d}'.format(n)
    format = 'NETCDF3_CLASSIC'
    last_time = str(time)[:12]
    filename = f'FOP_{FOP_Type}_{deviceID}_{sn}_{lon2}E_{lat2}N_{last_time}_Q.nc'
    file  = nc.Dataset(new_path+'/'+f'{filename}', 'w',format= format)

    # 创建维度
    time_dim = file.createDimension('num_time',1) 
    lon_dim = file.createDimension('num_longitude',1)
    lat_dim = file.createDimension('num_latitude',1)
    node_dim = file.createDimension('num_node',num_row)

    # 创建变量
    time_var = file.createVariable('time','f8',('num_time',))
    lon_var = file.createVariable('longitude','f4',('num_longitude',))
    lat_var = file.createVariable('latitude','f4',('num_latitude',))
        # 'Latitude': 这是新创建的变量的名称
        # 'f4': 这指定了变量的数据类型。'f8'代表64位浮点数（双精度）
        # ('Latitude',):指定新变量所依赖的维度，('Latitude',)创建元组，而 ('Latitude')为一个字符串
    profile_pressure_var = file.createVariable('profile_pressure','f4',('num_node',))
    profile_temperature_var = file.createVariable('profile_temperature','f4',('num_node',))
    profile_conductivity_var = file.createVariable('profile_conductivity','f4',('num_node',))
    seawater_salinity_var = file.createVariable('seawater_salinity','f4',('num_node',))
    seawater_density_var = file.createVariable('seawater_density','f4',('num_node',))
    seawater_sound_velocity_var = file.createVariable('seawater_sound_velocity','f4',('num_node',))

    # 添加属性
    time_var.long_name="the UTC time of current profile observed by the FOP"
    lon_var.long_name = 'the longitude of the deployed FOP' 
    lon_var.units = "degrees_east" 
    lat_var.long_name = 'the latitude of the deployed FOP'
    lat_var.units = "degrees_north"
    profile_pressure_var.long_name = "a profile of seawater pressure during ascending observed by the FOP"
    profile_pressure_var.units = "dbar"
    profile_temperature_var.long_name = "a profile of seawater temperature during ascending observed by the FOP"
    profile_temperature_var.units = "Celsius" 
    profile_conductivity_var.long_name = "a profile of seawater conductivity during ascending observed by the FOP"
    profile_conductivity_var.units = "mS/cm" 
    seawater_salinity_var.long_name = "a profile of seawater salinity during ascending during ascending calculated by the CTD"
    seawater_salinity_var.units = "PSU" 
    seawater_density_var.long_name =  "a profile of seawater density during ascending calculated by the CTD"
    seawater_density_var.units = "kg/m3" 
    seawater_sound_velocity_var.long_name = "a profile of seawater sound velocity during ascending calculated by the CTD"
    seawater_sound_velocity_var.units = "m/s" 
    
    # 变量中写入数据
        # 必须time_var[:] = time_data，不可以time_var= time_data
        # 使用createVariable方法创建一个变量后，这个变量实际上是一个特殊的对象，它代表netCDF文件中的一个数据区域。
        # time_var = time_data会改变time_var的引用，而不是修改它所代表的数据区域的内容
        # 相反，time_var[:]是用来引用该变量整个数据区域的一个“切片”
        # time_var[:] = time_data 实际上是将time_data数组中的数据复制到time_var所代表的数据区域中
    time_var[:] = time
    lon_var[:] = longitude
    lat_var[:] = latitude
    profile_pressure_var[:] = profile_pressure
    profile_temperature_var[:] = profile_temperature
    profile_conductivity_var[:] = profile_conductivity
    seawater_salinity_var[:] = seawater_salinity
    seawater_density_var[:] = seawater_density
    seawater_sound_velocity_var[:] = seawater_sound_velocity

    # 全局属性(需要放到最后来，不然添加不上全局属性)
    file.FOP_Platform = deviceID
    file.FOP_Type = FOP_Type 
    file.FOP_SN = sn
    file.FOP_ReleaseTime = ReleaseTime
    file.FOP_ReleaseLongitude = ReleaseLongitude
    file.FOP_ReleaseLatitude = ReleaseLatitude
    file.Sensor_accuracy = 'temperature,SBE41CP,±0.002℃;salinity,SBE41CP,±0.01m/s;pressure,SBE41CP,±0.8dbar/year'
    file.Comment = Comment
    file.Information = information
    file.Manufacturer = '中国科学院深海科学与工程研究所'
    file.Release_interval_time = f'{interval}h'
    file.Source = 'Wei Ma, TianJin University, wei.ma@tju.edu.cn'
    import datetime
    import pytz
    current_time = datetime.datetime.now() # 获取当前时间
    utc_time = current_time.astimezone(pytz.utc) # 将当前时间转换为 UTC 时间
    formatted_utc_time = utc_time.strftime("%Y-%m-%d %H:%M:%S %Z") # 格式化并显示 UTC 时间
    time_now = str(formatted_utc_time)[:19]
    file.History = f'Created by Python at {time_now} UTC'

    file.close() # 关闭文件
    nc_path = new_path+'/'+f'{filename}'
    return nc_path.replace('\\', '/')

In [6]:
file_path = r'C:\Users\lenovo\Desktop\FOP\FOP_FB_001_Pro003_202303270530.csv'
init_time = '2024-03-27 15:30:21'; interval = 2 
df = process_csv(file_path,init_time,interval)
# new_path = r'C:\Users\lenovo\Desktop\java\Glider05'
new_path = r'C:\Users\lenovo\Desktop\FOP' #'C:/Users/lenovo/Desktop/java/Glider05'  'C:/Users/lenovo/Desktop/任务帧nc数据'
longitude = 111.1234; latitude =23.1234
lon2 = 105.21 ;lat2 = 15.11
nc_generate(df,new_path,file_path,longitude,latitude,lon2,lat2,interval)

'C:/Users/lenovo/Desktop/FOP/FOP_FB_001_003_105.21E_15.11N_202403272130_Q.nc'